# Wide & Deep 模型
16年发布 用于分类和回归问题
应用到了Google Play中的应用推荐
[原始论文](https://arxiv.org/pdf/1606.07792v1.pdf)
## 稀疏特征
 * 离散值特征 性别 专业 等 只能在n个值中选则一个的信息
 * One-hot 
 * Eg:专业={计算机，人文，其他}。人文=[0,1,0] //词表
 * 叉乘 使用两个One-hot 数据进行叉乘（一一组合）
### 优点 
    有效 广泛用于工业界 广告点击预估 推荐算法
### 缺点 
    需要人工设计
    可能过拟合 所有特征都叉乘 相当于记住了每一个样本
    泛化能力差 没有出现过就不会起效果
## 密集特征
* 向量表达
    * Eg:词表={人工智能，你，他，慕课网}
    * 他=[0.3,0.2,0.6]
* Word2vec工具
### 优点
    带有语义信息 不同向量之间有相关性
    兼容没有出现过的特征组合
    更少人工参与
### 缺点
    过度泛化 推荐不怎么相关的产品
![wide wide&deep 模型对比](https://github.com/OoswantoO/tf-learn/blob/master/Tensorflow2.0EntryToAdvanced/2.11/截屏2021-03-02 上午11.08.33.jpg)
![wide&deep deep 模型对比](https://github.com/OoswantoO/tf-learn/blob/master/Tensorflow2.0EntryToAdvanced/2.11/截屏2021-03-02 上午11.09.57.jpg)
![google play 推荐算法模型图](https://github.com/OoswantoO/tf-learn/blob/master/Tensorflow2.0EntryToAdvanced/2.11/截屏2021-03-02 上午11.10.31.jpg)

使用tf.keras 实现 Wide&Deep模型
* 子类api 
* 功能api 
* 多输入与多输出



In [8]:
import os
import sys
import time

#画图
import matplotlib as mpl 
import matplotlib.pyplot as plt
#在notebook里画图
%matplotlib inline
import numpy as np
import sklearn
#处理数据
import pandas as pd

import tensorflow as tf
from tensorflow import keras

# print(tf.__version__)
# print(sys.version_info)
# for module in mpl,np,pd,sklearn,tf,keras:
#     print(module.__name__,module.__version__)

In [9]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()
print(housing.DESCR)
print(housing.data.shape)
print(housing.target.shape)

import pprint
pprint.pprint(housing.data[0:5])
pprint.pprint(housing.target[0:5]) 


.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bur

In [10]:
from sklearn.model_selection import train_test_split
x_train_all,x_test,y_train_all,y_test = train_test_split(housing.data,housing.target,random_state =7)
x_train,x_valid,y_train,y_valid = train_test_split(x_train_all,y_train_all,random_state = 11)
# print(x_valid.shape,y_valid.shape)
# print(x_train.shape,y_train.shape)
# print(x_test.shape,y_test.shape)
# print(np.max(x_train),np.min(x_train))

In [11]:
# x = (x-u均值)/ std方差
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)

In [12]:
print(np.max(x_train_scaled),np.min(x_train_scaled))

81.80221745207753 -2.3594398206242224


In [16]:
# 函数式api
input = keras.layers.Input(shape=x_train.shape[1:])
hidden1 = keras.layers.Dense(30,activation='relu')(input)
hidden2 = keras.layers.Dense(30,activation='relu')(hidden1)

concat = keras.layers.concatenate([input,hidden2])
output = keras.layers.Dense(1)(concat)

model = keras.models.Model(inputs = [input],outputs = [output])
model.summary()

model.compile(loss = "mean_sequared_error",optimizer = "sgd")


Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
dense_11 (Dense)                (None, 30)           270         input_5[0][0]                    
__________________________________________________________________________________________________
dense_12 (Dense)                (None, 30)           930         dense_11[0][0]                   
__________________________________________________________________________________________________
concatenate_3 (Concatenate)     (None, 38)           0           input_5[0][0]                    
                                                                 dense_12[0][0]             

ValueError: Unknown loss function:mean_sequared_error

In [14]:

callbacks = [
    keras.callbacks.EarlyStopping(patience=5,min_delta=1e-2)
]

history=model.fit(x_train_scaled,y_train,epochs=100,validation_data=(x_valid_scaled,y_valid),callbacks=callbacks)

Train on 11610 samples, validate on 3870 samples
Epoch 1/100
   32/11610 [..............................] - ETA: 3:38WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: 


InvalidArgumentError:  Received a label value of 5 which is outside the valid range of [0, 1).  Label values: 0 5 1 2 1 3 0 5 2 5 0 2 0 1 1 1 1 0 2 0 1 0 3 0 0 2 1 0 3 1 1 1
	 [[node loss/dense_7_loss/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at /Users/AllenWang/miniconda3/envs/tf3.7/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1751) ]] [Op:__inference_distributed_function_35725]

Function call stack:
distributed_function


In [ ]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=(8,5))
    plt.grid(True)
    plt.gca().set_ylim(0,3)
    plt.show()
    
plot_learning_curves(history)
#1. 参数众多 训练不充分
#2. 梯度消失 -> 链式法则 ->符合函数
#3. 批归一化 环节梯度消失

In [ ]:
model.evaluate(x_test_scaled,y_test)